<a href="https://colab.research.google.com/github/AnuragPamuru/dsc-180b-capstone-b03/blob/main/graphsage_combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from data_loader_sen import data_loader
from GraphSage import GraphSage

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
loader = data_loader("voting_features.csv", "tweets.csv", "edges.csv")
features, labels, A = loader.get_data()

In [47]:
model = GraphSage(A, features, labels, agg_func='MEAN', len_walk=13, num_neigh=10, F=1079)
acc = model.train_epoch(epochs=200, lr=1e-4)
# acc['acc']

Train length :70, Validation length :30
Epoch: 0
training loss 194.0902
Validtion: Average loss: 157.3358, Accuracy: 0.0000%
Epoch: 1
training loss 146.5895
Validtion: Average loss: 86.8538, Accuracy: 6.6667%
Epoch: 2
training loss 106.7988
Validtion: Average loss: 53.9038, Accuracy: 20.0000%
Epoch: 3
training loss 76.8291
Validtion: Average loss: 35.5670, Accuracy: 30.0000%
Epoch: 4
training loss 43.8500
Validtion: Average loss: 29.1897, Accuracy: 40.0000%
Epoch: 5
training loss 30.2012
Validtion: Average loss: 15.4366, Accuracy: 63.3333%
Epoch: 6
training loss 18.6025
Validtion: Average loss: 17.6969, Accuracy: 50.0000%
Epoch: 7
training loss 18.7169
Validtion: Average loss: 18.6164, Accuracy: 53.3333%
Epoch: 8
training loss 15.2994
Validtion: Average loss: 20.6304, Accuracy: 50.0000%
Epoch: 9
training loss 20.7981
Validtion: Average loss: 22.1648, Accuracy: 56.6667%
Epoch: 10
training loss 15.6459
Validtion: Average loss: 10.8146, Accuracy: 73.3333%
Epoch: 11
training loss 13.3071
V

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import random

"""
Set of modules for aggregating embeddings of neighbors.
"""

class MeanAggregator(nn.Module):
    """
    Aggregates a node's embeddings using mean of neighbors' embeddings
    """
    def __init__(self, features, cuda=False, gcn=False): 
        """
        Initializes the aggregator for a specific graph.
        features -- function mapping LongTensor of node ids to FloatTensor of feature values.
        cuda -- whether to use GPU
        gcn --- whether to perform concatenation GraphSAGE-style, or add self-loops GCN-style
        """

        super(MeanAggregator, self).__init__()

        self.features = features
        self.cuda = cuda
        self.gcn = gcn
        
    def forward(self, nodes, to_neighs, num_sample=10):
        """
        nodes --- list of nodes in a batch
        to_neighs --- list of sets, each set is the set of neighbors for node in batch
        num_sample --- number of neighbors to sample. No sampling if None.
        """
        # Local pointers to functions (speed hack)
        _set = set
        if not num_sample is None:
            _sample = random.sample
            samp_neighs = [_set(_sample(to_neigh, 
                            num_sample,
                            )) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]
        else:
            samp_neighs = to_neighs

        if self.gcn:
            samp_neighs = [samp_neigh | set([nodes[i]]) for i, samp_neigh in enumerate(samp_neighs)]
        
        unique_nodes_list = list(set.union(*samp_neighs))
        unique_nodes = {n:i for i,n in enumerate(unique_nodes_list)}
        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]   
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
        mask[row_indices, column_indices] = 1
        
        if self.cuda:
            mask = mask.cuda()
        
        num_neigh = mask.sum(1, keepdim=True)
        mask = mask.div(num_neigh)
        if self.cuda:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
        to_feats = mask.mm(embed_matrix)
        return to_feats

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    """
    Encoder for GraphSAGE implementation.
    """
    def __init__(self, features, feature_dim, 
            embed_dim, adj_lists, aggregator,
            num_sample=10,
            base_model=None, gcn=False, cuda=False, 
            feature_transform=False): 
        super(Encoder, self).__init__()

        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        if base_model != None:
            self.base_model = base_model

        self.gcn = gcn
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        self.weight = nn.Parameter(
                torch.FloatTensor(embed_dim, self.feat_dim if self.gcn else 2 * self.feat_dim))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, nodes):
        """
        Generates embeddings for a batch of nodes.
        """
        neigh_feats = self.aggregator.forward(nodes, [self.adj_lists[int(node)] for node in nodes], 
                self.num_sample)
        
        if not self.gcn:
            if self.cuda:
                self_feats = self.features(torch.LongTensor(nodes).cuda())
            else:
                self_feats = self.features(torch.LongTensor(nodes))
            combined = torch.cat([self_feats, neigh_feats], dim=1)
        else:
            combined = neigh_feats
        combined = F.relu(self.weight.mm(combined.t()))
        return combined

In [ ]:
from math import sqrt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

class GraphConvolution(nn.Module):
    """Vanilla GCN Layer."""

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        # multiply input by weight
        support = torch.mm(input, self.weight)
        # multiply adjacency matrix by weighted product 
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class LPAConvolution(nn.Module):
    """
    GCN LPA Layer.
    """

    def __init__(self, in_features, out_features, adj, bias=True):
        super(LPAConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
        
        self.adjacency_mask = Parameter(adj.clone()).to_dense()

    def reset_parameters(self):
        stdv = 1. / sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x, adj, y):
        adj = adj.to_dense()
        
        # W * x
        support = torch.mm(x, self.weight)
        
        # Hadamard Product: A' = Hadamard(A, M)
        adj = adj * self.adjacency_mask
        
        # Row-Normalize: D^-1 * (A')
        adj = F.normalize(adj, p=1, dim=1)

        # output = D^-1 * A' * X * W
        output = torch.mm(adj, support)
        
        # y' = D^-1 * A' * y
        y_hat = torch.mm(adj, y)

        if self.bias is not None:
            return output + self.bias, y_hat
        else:
            return output, y_hat

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


In [ ]:
import numpy as np
import scipy.sparse as sp
import torch
from scipy.linalg import fractional_matrix_power as matrix_frac_power
import matplotlib.pyplot as plt
import os
import networkx as nx

def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.
    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.
    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes)
    return y[labels]


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    """Returns the accuracy given prediction outputs and target labels.

    Args:
        output: model predictions
        labels: one hot encoded labels

    Returns:
        Accuracy metric of the model
    """
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


def plot_stats(outdir, model_name, training_losses, val_losses):
    e = len(training_losses)
    x_axis = np.arange(1, e + 1, 1)
    plt.figure()
    plt.plot(x_axis, training_losses, label="Training Loss")
    plt.plot(x_axis, val_losses, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.legend(loc='best')
    plt.title(model_name + " Loss Plot")
    if not os.path.exists('data/out'):
        os.makedirs('data/out')
    plt.savefig("{}_loss_plot.png".format(outdir + model_name))
    
    
def draw(outdir, outputname, adj, predictions, labels):
    G = nx.from_numpy_matrix(adj.to_dense().detach().numpy(), nx.DiGraph())
    plt.figure(figsize=(10,10))
    nx.draw(G, node_size=10, edge_size=1,node_color=predictions.detach().numpy(), cmap = 'tab10')
    plt.savefig(f"{outdir}{outputname}.png")

        
def plotGCN(model, adj, features, labels, outdir, outputname):
    model.eval()
    output = model(features, adj)
    predictions = output.argmax(dim = 1)
    draw(outdir, outputname, adj, predictions, labels)
    
    
def plotGCNLPA(model, adj, features, labels, outdir, outputname):
    output, _ = model(features, adj, labels)
    predictions = output.argmax(dim = 1)
    draw(outdir, outputname, adj, predictions, labels)
    
    
def plotGraphSAGE(model, adj, adj_lists, features, labels, outdir, outputname):
    model.eval()
    output = model(torch.LongTensor(range(labels.shape[0])))
    predictions = output.argmax(dim = 1)
    draw(outdir, outputname, adj, predictions, labels)
     

In [ ]:
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import random

import torch
import torch.nn.functional as F
import torch.optim as optim

# from models import *
# from encoders import *
# from aggregators import *
# from utils import accuracy, one_hot_embedding, plot_stats
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F

# from layers import *


def train_GraphSAGE(model, optimizer, adj, features, labels, idx_train, idx_val, epoch, fastmode = False):

    batch_nodes = idx_train
    start_time = time.time()
    
    # Set model to training mode
    model.train()
    
    optimizer.zero_grad()
    loss_train, train_output = model.loss(batch_nodes, 
            Variable(torch.LongTensor(labels[batch_nodes])))
    
    acc_train = accuracy_score(labels[batch_nodes], train_output.data.numpy().argmax(axis=1))    
    loss_train.backward()
    optimizer.step()
    
    loss_val, val_output = model.loss(idx_val, Variable(torch.LongTensor(labels[idx_val])))
    acc_val = accuracy_score(labels[idx_val], val_output.data.numpy().argmax(axis=1))
    print('Epoch: {:04d}'.format(epoch + 1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - start_time))
    
    return loss_train.item(), loss_val.item() 
    
def test_GraphSAGE(model, adj, features, labels, idx_test):
    model.eval()
    loss_test, test_output = model.loss(idx_test, Variable(torch.LongTensor(labels[idx_test])))
    acc_test = accuracy_score(labels[idx_test], test_output.data.numpy().argmax(axis=1))
    print("Test:",
          "loss: {:.4f}".format(loss_test.item()),
          "acc: {:.4f}".format(acc_test.item()))
    
    
def model_build(name, adj, features, labels, idx_train, idx_val, idx_test, outdir,
                no_cuda = True, fastmode = False, seed = 42, epochs = 50, learning_rate = 0.1,
                weight_decay = 5e-4, hidden = 16, dropout = 0.5, lpa_weight_decay = 0, sample1 = 25, sample2 = 10, gcn_mode=True):
    
    """Builds and trains the model given data and model parameters."""
    
    cuda = (not no_cuda) and torch.cuda.is_available()
    if cuda and name == 'GraphSAGE': 
        print("CUDA not available for GraphSAGE...")
        cuda = False
    
    if cuda: print("Using CUDA...")
    
    # set seed
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda: torch.cuda.manual_seed(seed)
    
    # Set device
    device = 'cuda' if cuda else 'cpu'
    torch.device(device)
    
    # Load data
    if name in ['GraphSAGE']:
        n_feats = features.shape[1]
        n_classes = len(np.unique(labels))
        new_features = nn.Embedding(*features.shape)
        new_features.weight = nn.Parameter(torch.FloatTensor(features), requires_grad=False)
        features = new_features
        
    features = features.to(device)
    if name in ['GCN', 'GCNLPA']:
        adj = adj.to(device)
    labels = labels.to(device)
    idx_train = idx_train.to(device)
    idx_val = idx_val.to(device)
    idx_test = idx_test.to(device)
    
    lpa_labels = None
    
    # Create model
        
    
    if name == "GraphSAGE": 
        
        if not gcn_mode:
            agg1 = MeanAggregator(features, cuda=False)
            enc1 = Encoder(features, n_feats, hidden, adj, agg1, gcn=False, cuda=False)
            agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
            enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, 128, adj, agg2, base_model=enc1, gcn=False, cuda=False)
        else:
            agg1 = MeanAggregator(features, cuda=True)
            enc1 = Encoder(features, n_feats, hidden, adj, agg1, gcn=True, cuda=False)
            agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
            enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, hidden, adj, agg2,
                    base_model=enc1, gcn=True, cuda=False)
        
        enc1.num_sample = sample1
        enc2.num_sample = sample2
        
        model = SupervisedGraphSage(n_classes, enc2)
        model.to(device)
        
        optimizer = optim.SGD(filter(lambda p : p.requires_grad, model.parameters()), lr = learning_rate, weight_decay = weight_decay)
        
        total_time = time.time()
        print("Training {} model for {} epochs...".format(name, epochs))
        train_losses = []
        val_losses = []
        
        for e in range(epochs):
            train_loss, val_loss = train_GraphSAGE(model, optimizer, adj, features, labels, idx_train, idx_val, epoch = e, fastmode=fastmode)
            train_losses.append(train_loss)
            val_losses.append(val_loss)
        print("Finished. Total time elapsed: {:.4f}s".format(time.time() - total_time))
        
        # plot_stats(outdir, "GraphSAGE", train_losses, val_losses)
        
        test_GraphSAGE(model, adj, features, labels, idx_test)
        
        return model
class SupervisedGraphSage(nn.Module):
    def __init__(self, nclass, enc):
        super(SupervisedGraphSage, self).__init__()
        self.xent = nn.CrossEntropyLoss()
        self.enc = enc
        self.weight = nn.Parameter(torch.FloatTensor(nclass, enc.embed_dim))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, nodes):
        embeds = self.enc(nodes)
        scores = self.weight.mm(embeds)
        return scores.t()
    
    def loss(self, nodes, labels):
        scores = self.forward(nodes)
        if list(labels.size()) == [1]: # incase the batch only contains 1 row...
            return self.xent(scores, labels), scores
        return self.xent(scores, labels.squeeze()), scores


        


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# from layers import *
   
    
class SupervisedGraphSage(nn.Module):
    def __init__(self, nclass, enc):
        super(SupervisedGraphSage, self).__init__()
        self.xent = nn.CrossEntropyLoss()
        self.enc = enc
        self.weight = nn.Parameter(torch.FloatTensor(nclass, enc.embed_dim))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, nodes):
        embeds = self.enc(nodes)
        scores = self.weight.mm(embeds)
        return scores.t()
    
    def loss(self, nodes, labels):
        scores = self.forward(nodes)
        if list(labels.size()) == [1]: # incase the batch only contains 1 row...
            return self.xent(scores, labels), scores
        return self.xent(scores, labels.squeeze()), scores


In [ ]:
import os
import torch
import numpy as np
import scipy.sparse as sp
import networkx as nx
import matplotlib.pyplot as plt
from utils import one_hot_embedding, normalize, sparse_mx_to_torch_sparse_tensor, encode_onehot
from scipy.linalg import fractional_matrix_power as matrix_frac_power
from collections import defaultdict

def load_data(outdir, path="", dataset="cora", nodefile="cora.content", edgefile="cora.cites", symm_norm=False, ratio_train = 0.05, ratio_val = 0.20, ratio_test = 0.40, shuffle = False, seed = 42):
    
    print('Loading {} dataset from {}...'.format(dataset, path))

    print("Parsing node feature and label data...")
    # Load feature and label data into a numpy array
    idx_features_labels = np.genfromtxt("{}{}".format(path, nodefile),
                                        dtype=np.dtype(str))
    
    # Put features into a sparse matrix, exclude first and last columns
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    
    feat_data = idx_features_labels[:, 1:-1].astype(float)
    
    # Onehot encode labels
    labels = encode_onehot(idx_features_labels[:, -1])
        
    # Get node id
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    
    # Map index to node id
    idx_map = {j: i for i, j in enumerate(idx)}
    
    print("Parsing edge data...")
    # Load edge data into a numpy array
    edges_unordered = np.genfromtxt("{}{}".format(path, edgefile),
                                    dtype=np.int32)
    
    # Create list of edges encoded into indices using index map
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)

    adj_lists = defaultdict(set)
    for i in edges:
        adj_lists[i[0]].add(i[1])
        adj_lists[i[1]].add(i[0])
    
    # Create adjacency matrix
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # Build symmetric adjacency matrix    
    if not symm_norm:
        adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
        adj = normalize(adj + sp.eye(adj.shape[0]))
    else:
        # use the D^-1/2AD^-1/2 formula
        adj = adj + sp.eye(adj.shape[0]) # add self loop
        D = np.diag(np.array(adj.sum(axis=1)).flatten()) # build degree matrix
        D_prime = matrix_frac_power(D,-0.5)
        D_prime = sp.coo_matrix(D_prime, shape=(adj.shape[0],adj.shape[0]),dtype=np.float32) # convert to sparse format
        adj = D_prime @ adj @ D_prime # compute the normalized symmetric version

    # Normalize features
    features = normalize(features)
    
    # save features 
    # if not os.path.exists(outdir):
    #     os.makedirs(outdir)
    # with open(outdir + 'features.npz', 'wb+') as fh:
    #     sp.save_npz(fh, features, compressed=False)
    # # save adj
    # with open(outdir + 'adjmatrix.npz', 'wb+') as fh:
    #     sp.save_npz(fh, adj, compressed=False)
    # # save labels
    # with open(outdir + 'labels.txt', 'w+') as fh:
    #     np.savetxt(fh, labels)
     
    # Convert features, labels, and adjacency matrix to Tensors
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)
    
    idx_train, idx_val, idx_test = data_split(labels.size()[0], ratio_train = ratio_train, ratio_val = ratio_val, ratio_test = ratio_test, shuffle = shuffle, seed = seed)

    return adj, adj_lists, features, feat_data, labels, idx_train, idx_val, idx_test

def data_split(n, ratio_train = 0.05, ratio_val = 0.20, ratio_test = None, shuffle = False, seed = 42):
    """Splits n data into train, validation and test splits by indices."""
    
    if ratio_test is None or ratio_test > (1 - ratio_train - ratio_val):
        ratio_test = 1 - ratio_train - ratio_val
    
    if shuffle:
        np.random.seed(seed)
        indices = np.random.permutation(n)
    else:
        indices = range(n)
    
    # Train-Validation-Test Split
    n_train = int(n * ratio_train)
    n_val = int(n * ratio_val)
    n_test = int(n * ratio_test)

    idx_train = indices[:n_train]
    idx_val = indices[n_train: n_train + n_val]
    idx_test = indices[n_train + n_val: n_train + n_val + n_test]
    
    # Convert split indices to Tensors
    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return idx_train, idx_val, idx_test




In [ ]:
adj, adj_lists, features, feat_data, labels, idx_train, idx_val, idx_test = load_data("")

Loading cora dataset from ...
Parsing node feature and label data...
Parsing edge data...


In [ ]:
len(idx_test)

1083

In [ ]:
GraphSAGE_model = model_build("GraphSAGE", adj_lists, feat_data, labels, idx_train, idx_val, idx_test, outdir="")

Training GraphSAGE model for 50 epochs...
Epoch: 0001 loss_train: 1.9419 acc_train: 0.1185 loss_val: 1.9359 acc_val: 0.1608 time: 0.2584s
Epoch: 0002 loss_train: 1.9415 acc_train: 0.1037 loss_val: 1.9333 acc_val: 0.1682 time: 0.1429s
Epoch: 0003 loss_train: 1.9388 acc_train: 0.1185 loss_val: 1.9310 acc_val: 0.1904 time: 0.2216s
Epoch: 0004 loss_train: 1.9346 acc_train: 0.1704 loss_val: 1.9298 acc_val: 0.2384 time: 0.1416s
Epoch: 0005 loss_train: 1.9323 acc_train: 0.2000 loss_val: 1.9281 acc_val: 0.2144 time: 0.1549s
Epoch: 0006 loss_train: 1.9300 acc_train: 0.2074 loss_val: 1.9256 acc_val: 0.2754 time: 0.1502s
Epoch: 0007 loss_train: 1.9282 acc_train: 0.2370 loss_val: 1.9249 acc_val: 0.2717 time: 0.1465s
Epoch: 0008 loss_train: 1.9257 acc_train: 0.2889 loss_val: 1.9224 acc_val: 0.2994 time: 0.1464s
Epoch: 0009 loss_train: 1.9221 acc_train: 0.3407 loss_val: 1.9187 acc_val: 0.3124 time: 0.1487s
Epoch: 0010 loss_train: 1.9198 acc_train: 0.3778 loss_val: 1.9190 acc_val: 0.3142 time: 0.1612